
#  <span style="color:#1893F8">Implementing SQL queries using Pandas</span>
I really hope you don't mind that all my commentary is in English. :) 


## <span style="color:#1893F8">Project description</span>
The aim of the project is to implement 7 given SQL queries in Python using Pandas.  
Each query has a short description of what it's doing and details about my approach in doing the same in 
Pandas.   
The resoult is accepted when SQL and Pandas give the same data with eventual difference in permutation of rows.
  
In every query, as first I present SQL code that I received and right after it there is my approach of doing the same but in Pandas.

I am working on a public database of Travel Stack Exchange. https://travel.stackexchange.com/  
Details about data stes can be foun here. https://ia800107.us.archive.org/27/items/stackexchange/readme.txt


## <span style="color:#1893F8"> Testing resoults</span>

I decided to use two methods to check if my resoults are fine.

### <span style="color:#999DA1">np.equal</span>  

First is  *np.equal(DF1,DF2)* which gives as a resoult a data table with the same size as DF1 and DF2 filled with True values if the same cell of a data frame has the same value in both DF1 and DF2, and False otherwise.   
It sometimes showes single Falses which is metter of None!=NaN or NaN!=Nan. From Pandas point of view there's no difference so I ignore that fact.  

  
### <span style="color:#999DA1"> pd.testing.assert_frame_equal  </span> 

Second method is *pd.testing.assert_frame_equal(DF1,DF2)* which compares two data frames. Troubles with this method occur when compared data differes in row permutation(check_like=True dosen't fix it).  
It returns list of errors and detailed description of what's wrong. But It's rather unpleasent to read, so sometimes I skip this method.  
If it happens it means that my data frame differs in permutation of rows and it's unclear for me how to change it. It can be seen in *np.equal*.  
Boring part of this method is that if everything is ok the function returns nothing. 
So if it's effect is unnoticable, then data sets are identical. 

I am working on Windows but only becouse I have Jupyter Lab already downloaded here. 



## Base two libraries that I am going to use

In [1]:
import pandas as pd  ## For handling data frames
import numpy as np   ## For various useful functions and structures

## Loading gziped data sets into global env

In [2]:
import os, os.path   # Library used only for loading data


## Importing data sets making them Data Frames in Pandas

path_to_data="C:\\Users\\Bartek\\PycharmProjects\\studia\\venv\\Lib\\PDU\\data\\"    # This path works only on my pc

files=["Badges","Comments","PostLinks","Posts","Tags","Users","Votes"] ## Names of all the data sets that I'll be using

for file in files:
    globals()[file] = pd.read_csv(     
        os.path.join(path_to_data+file+".csv.gz"),
                       comment="#", compression="gzip")
    
    



## Putting data sets into SQLite database

In [3]:
import tempfile, sqlite3   # Library used only for loading data into SQLite

database= os.path.join(tempfile.mkdtemp(), "Travel.db") ## Creating SQLite database named "Travel.db"
if os.path.isfile(database):
    os.remove(database)

connection=sqlite3.connect(database)     # Creating connection to the database


                                         # Uploading all the data sets to database 
Badges.to_sql("Badges",connection)
Comments.to_sql("Comments",connection)
PostLinks.to_sql("PostLinks",connection)
Posts.to_sql("Posts",connection)
Tags.to_sql("Tags",connection)
Users.to_sql("Users",connection)
Votes.to_sql("Votes",connection)


# <span style="color:#1893F8"> Query nr 1</span>

### <span style="color:#1893F8"></span> What this query does? 

The goal is to find 10 people whose posts were added to favorites the most and extract their *DisplayName, Age, Location*, sum of favourites as *FavoriteTotal*, title of most favourited question as *MostFavoriteQuestion*, maximal amount of favorites that post from that person received as  *MostFavouriteQuestionLikes*. The receved data table is sorted decreasing by *FavouriteTotal*.

### <span style="color:#1893F8"></span> How do I approach it? 

First I want to calculate the sum (FavoriteTotal) and maximum amunt of favourites (MostFavoriteQuestionLikes) that every person received for their questions and leave only 10 people with highes FavoriteTotal. Then by using their *OwnerUserId*  and *MostFavoriteQuestionLikes* I find *Titles* of most favourited posts of every person. Then again by using *OwnerUserId* I extract *DisplayName, Age, Location* from *Users* data set. The rest is to clean the data frame of unimportant data and sort it by *FavoriteTotal*.

## <span style="color:#999DA1">SQL Query</span>

In [4]:
DF1_sql=pd.read_sql_query("""   SELECT
                            Users.DisplayName,
                            Users.Age,
                            Users.Location,
                            SUM(Posts.FavoriteCount) AS FavoriteTotal,
                            Posts.Title AS MostFavoriteQuestion,
                            MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                        FROM Posts
                        JOIN Users ON Users.Id=Posts.OwnerUserId
                        WHERE Posts.PostTypeId=1
                        GROUP BY OwnerUserId
                        ORDER BY FavoriteTotal DESC
                        LIMIT 10
                  """, connection)



## <span style="color:peru">Pandas query</span>


In [33]:
sum_and_max=Posts.groupby("OwnerUserId").FavoriteCount.agg([np.max,np.sum]).sort_values("sum",ascending=False).head(10)  ## getting top ten FavoriteCounts with OwnerUserId and MostFavoriteQuestionLikes 




sum_max_titles=sum_and_max.merge(Posts.loc[:,("Title","OwnerUserId","FavoriteCount")],    ## merging OwnerUserId to his/hers post with MostFavoriteQuestionLikes and getting it's Title
                          left_on       =       ("OwnerUserId","amax"),
                          right_on      =       ("OwnerUserId","FavoriteCount"))

resoults=Users.loc[:,("DisplayName","Age","Location","Id")].merge(                      ## Getting additional data obout all 10 selected users
                  sum_max_titles,
                  left_on       =       "Id",
                  right_on      =       "OwnerUserId")

DF1_pd=resoults.loc[:,("DisplayName","Age","Location","sum","Title","FavoriteCount")].sort_values("sum",ascending=False).reset_index(drop=True)    ## cleaning the resoult data frame and sorting it by  FavoriteCounts


DF1_pd.columns=("DisplayName","Age","Location","FavoriteTotal","MostFavoriteQuestion","MostFavoriteQuestionLikes")

## Comparison

In [36]:

pd.testing.assert_frame_equal(DF1_sql,DF1_pd)
np.equal(DF1_sql,DF1_pd)  # all Falses come from differences betwen NaNs and None!= NaN


,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,True,True,True,True,True,True
1,True,False,True,True,True,True
2,True,False,False,True,True,True
3,True,True,True,True,True,True
4,True,False,False,True,True,True
5,True,False,True,True,True,True
6,True,True,True,True,True,True
7,True,True,True,True,True,True
8,True,False,False,True,True,True
9,True,False,True,True,True,True


# <span style="color:#1893F8"> Query nr 2</span>

### What this query does? 

The goal of this query is to extract *Tilte* and *Id* of 10 posts with the highest number of answers. 

### How do I approach it? 

I'm going to group posts by ParentId (posts with the same ParentId belong to the same questions), but I have to filter them with *PostTypeId==2* becouse only those posts are answers and then I count how much answers each post has. Then I sort the posts decreasing by their count and leave only top 10. I'm left with ParentId's of 10 posts with highest answer count. The only thing left is to connect them with Titles. So I extract *Id* and *Title* of all Posts to merge the ParentId's with Id's. After that I'm left with data frame that I wanted. To get exactly the same data frame as SQL gives I have to sort Count decreasing and Id ascending. 

## <span style="color:#999DA1">SQL Query</span>

In [8]:
DF2_sql=pd.read_sql_query("""   SELECT
                            Posts.ID,
                            Posts.Title,
                            Posts2.PositiveAnswerCount
                        FROM Posts
                        JOIN (
                                SELECT
                                    Posts.ParentID,
                                    COUNT(*) AS PositiveAnswerCount
                                FROM Posts
                                WHERE Posts.PostTypeID=2 AND Posts.Score>0
                                GROUP BY Posts.ParentID
                            ) AS Posts2
                            ON Posts.ID=Posts2.ParentID
                        ORDER BY Posts2.PositiveAnswerCount DESC
                        LIMIT 10  
                  """, connection)


## <span style="color:peru">Pandas query</span>


In [37]:
                    ## only answers       with score >0 | gruped by "ParentId" | COUNT()| sort to get 10 with highest sum | converting series to data frame
Posts2=Posts.loc[((Posts.PostTypeId==2)&(Posts.Score>0)),:].groupby("ParentId").size().sort_values(ascending=False).head(10).to_frame()


Titles=Posts.loc[:,("Id","Title")]  ## selecting titles and Id of posts i could give filter it with PostTypeId but i don't think it would make it faster

DF2_pd=Posts2.merge(Titles,                                ## merging data frames
                      left_on="ParentId",
                      right_on="Id",
                      how="left").reindex(columns=("Id","Title",0))

DF2_pd.columns=("Id","Title","PositiveAnswerCount")      ## chaning the columns names
DF2_pd.sort_values(["PositiveAnswerCount","Id"],ascending=[False,True],inplace=True) ## Special sort to get same resoults as SQL

## Comparison

In [38]:

#pd.testing.assert_frame_equal(DF2_sql,DF2_pd)   # throws an error but both data frames are literally identical
np.equal(DF2_sql,DF2_pd)

,Id,Title,PositiveAnswerCount
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True
7,True,True,True
8,True,True,True
9,True,True,True


# <span style="color:#1893F8"> Query nr 3</span>

### What this query does? 

This query extracts *Titles* of the posts (questions) which had the most up votes (not counting down votes) in every year since 2011 up to 2017.  
It returnes a data frame with *Title* of the post, next to it *Year* of publishing the post and number of it's up votes as *Count*. 

### How do I approach it? 

So in *Votes* I look for number of votes with *VotetypeId==1* in evety post which is a question (has a PostTypeId==1) and a change it's *CreationDate* to new format %yyyy.  
Then I look for maximum named *Count* in each *Year* and the *PostId* of post which it comes from. At last we have to get the *Title* of post by using fount *PostId*.  
We do it while merging previous data to get the final resoult. 
## <span style="color:#999DA1">SQL Query</span>

In [11]:
DF3_sql=pd.read_sql_query("""  
                    SELECT
                        Posts.Title,
                        UpVotesPerYear.Year,
                        MAX(UpVotesPerYear.Count) AS Count
                    FROM (
                            SELECT
                                PostId,
                                COUNT(*) AS Count,
                                STRFTIME('%Y', Votes.CreationDate) AS Year
                            FROM Votes
                            WHERE VoteTypeId=2
                            GROUP BY PostId, Year
                        ) AS UpVotesPerYear
                    JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
                    WHERE Posts.PostTypeId=1
                    GROUP BY Year
                  """, connection)


## <span style="color:peru">Pandas query</span>


In [12]:


YearUpVotes=Votes.loc[(Votes.VoteTypeId==2) & (Votes.PostId.isin(Posts.loc[Posts.PostTypeId==1, "Id"])),("PostId","CreationDate")]

YearUpVotes["CreationDate"]=pd.to_datetime(YearUpVotes.CreationDate).dt.strftime("%Y") ## Chaging the date format from %dd %mm % yyyy to %yyyy




UpVotesPerYear=YearUpVotes.groupby(["PostId","CreationDate"]).size().reset_index().rename(index=str,columns={"CreationDate":"Year",0:"Count"})

UpVotesPerYear2=UpVotesPerYear.groupby("Year",as_index=False)["Count"].max().merge(UpVotesPerYear,on=["Year","Count"])


##
DF3_pd=UpVotesPerYear2.merge(Posts.loc[Posts.PostTypeId==1,("Id","Title")],
                     left_on="PostId",
                     right_on="Id",how="inner").loc[:,("Title","Year","Count")]


## Comparison

In [13]:

pd.testing.assert_frame_equal(DF3_sql,DF3_pd)
np.equal(DF3_sql,DF3_pd)


,Title,Year,Count
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True


# <span style="color:#1893F8">Query nr 4</span>

### What this query does? 

In this query we are looking for the answers to the posts with the biggest *Difference* between the maximum score of any answer to the question and it's own score.  
I accept only those post which have answer with *Difference* bigger than 50  for those I and extract *Id, Title*, highest score on any answer as *MaxScore*, score of the answer as *AcceptedScore* and the *Difference*.  
Resoult data frame is sorted by *Difference* decreasing.

### How do I approach it? 

First I extract answers to the posts (those with PostTypeId==2), then after grouping them by "ParentId" I look for the maximum value in each group I name it *MaxScore*.  
Next I calculate the *Difference* and leave only the answers with it bigger than 50. Next I join resoulted answers using *ParentId* and *Id*,  
with their oryginal questions and extract the question *Id, Title* and also answer's *MaxScore, AcceptedScore* nad *Difference*.  
Resoulted data frame is then sorted by the *Difference* decreasing.

## <span style="color:#999DA1">SQL Query</span>

In [14]:
DF4_sql=pd.read_sql_query("""  
                    SELECT
                        Questions.Id,
                        Questions.Title,
                        BestAnswers.MaxScore,
                        Posts.Score AS AcceptedScore,
                        BestAnswers.MaxScore-Posts.Score AS Difference
                    FROM (
                            SELECT Id, ParentId, MAX(Score) AS MaxScore
                            FROM Posts
                            WHERE PostTypeId==2
                            GROUP BY ParentId
                        ) AS BestAnswers
                    JOIN (
                            SELECT * FROM Posts
                            WHERE PostTypeId==1
                        ) AS Questions
                        ON Questions.Id=BestAnswers.ParentId
                    JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
                    WHERE Difference>50
                    ORDER BY Difference DESC
                  """, connection)

## <span style="color:peru">Pandas query</span>

In [39]:
RightPosts=Posts.loc[Posts.PostTypeId==2,("ParentId","Id","Score")].groupby(["ParentId"])


#ba stands for Best Answers
ba1=RightPosts.Score.agg([np.max]).reset_index(drop=False) # calculating the Max Score
ba2=RightPosts.head(30)  # number inside head() is a limit of returned rows in in a group. it has to be high so no data is lost. There's propably a better way of doing it that I'm not aware of



ba=ba1.merge(ba2,   ## conecting Max Score to rest nessesery data
            on="ParentId").rename(index=str,columns={"amax":"MaxScore","Score":"AcceptedScore"}) # renaming the columns, "amax" was the resoult of aggregate(np.max)

ba["Difference"]=ba.MaxScore-ba.AcceptedScore  # creating Diffrence column


ba=ba.loc[ba.Difference>50,:]                 # filtering by difference

resoults=pd.merge(ba,
                  Posts.loc[Posts.PostTypeId==1,("Id","Title","AcceptedAnswerId")],    # mergigng data sets to get expected resoults
                       left_on=["ParentId","Id"],
                       right_on=["Id","AcceptedAnswerId"]).rename(index=str,columns={"Id_y":"Id"})

DF4_pd=resoults.loc[:,("Id","Title","MaxScore","AcceptedScore","Difference")].sort_values("Difference",ascending=False).reset_index(drop=True)   ## getting only required data | drop=True to get rid of unimportant index


## Comparison

In [40]:

pd.testing.assert_frame_equal(DF4_sql,DF4_pd,check_like=True)
np.equal(DF4_sql,DF4_pd)


,Id,Title,MaxScore,AcceptedScore,Difference
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
5,True,True,True,True,True
6,True,True,True,True,True
7,True,True,True,True,True


# <span style="color:#1893F8">Query nr 5</span> 

### What this query does? 

This query extracts *Posts* (Questions) which have the biggest sum of Scores in all it's comments. It returns the 10 posts with *Title* and sum of scores of it's comments, sorted with sum decreasing. 

### How do I approach it? 

First I calculate the sum (CommentsTotalScore) of the Scores of the comments. Then I select the posts with *PostTypeId==1* which are simply questions and get their *OwnerUserId, Id, Title*.  
After that I merge those two by using *PostId, UserId* from comments and *Id, OwnerUserId* from Posts.  
Next I sort merged data frame by *CommentsTotalScore and Id* (CommentsTotalScore is enough but adding Id to sorting gives me the same permutation as SQL) and take only top 10 resoults. 

## <span style="color:#999DA1">SQL Query</span>

In [17]:
DF5_sql=pd.read_sql_query(""" 
                    SELECT
                        Posts.Title,
                        CmtTotScr.CommentsTotalScore
                    FROM (
                            SELECT
                                PostID,
                                UserID,
                                SUM(Score) AS CommentsTotalScore
                            FROM Comments
                            GROUP BY PostID, UserID
                        ) AS CmtTotScr
                    JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
                    WHERE Posts.PostTypeId=1
                    ORDER BY CmtTotScr.CommentsTotalScore DESC
                    LIMIT 10
                  """, connection)

## <span style="color:peru">Pandas query</span>

In [18]:
CmtTotScr=Comments.groupby(["PostId","UserId"]).Score.agg([np.sum])


posts_select=Posts.loc[Posts.PostTypeId==1,("OwnerUserId","Id","Title")] ## selecting needed columns from posts that are questions

DF5_pd=posts_select.merge(  CmtTotScr,                                   ## connecting sums to the posts that they reffer to  
                            left_on=["Id","OwnerUserId"],
                            right_on=["PostId","UserId"]).sort_values(["sum","Id"],  ## sorting to get questions with highest sum, and also by Id to get the same resoult as in SQL
                                                                      ascending=[False,True]).loc[:,("Title","sum")] ##extracting only needed data

DF5_pd=DF5_pd.rename(index=str,columns={"sum":"CommentsTotalScore"}).head(10).reset_index(drop=True)                 ## renaming and cutting resoult to 10 rows

## Comparison

In [19]:

pd.testing.assert_frame_equal(DF5_sql,DF5_pd,check_like=True)
np.equal(DF5_sql,DF5_pd)


,Title,CommentsTotalScore
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,True


# <span style="color:#1893F8"> Query nr 6</span>

### What this query does? 

This query extracts *Id, DisplayName, Reputation, Age, Location* of users which have gold badge (which means badge with *Class==1*) and only 2 to 10 of all users has it.  

### How do I approach it? 

I am going to be looking for unique *Badges* from *Class==1*. I group them by *Name* and leave only those wich occur more than once but no more than ten times.  
Next I check *UserId* of their owners and by using it as my key I extract wanted data about those people from data set *Users*.


## <span style="color:#999DA1">SQL Query</span>

In [20]:
DF6_sql=pd.read_sql_query(""" 
                    SELECT DISTINCT
                        Users.Id,
                        Users.DisplayName,
                        Users.Reputation,
                        Users.Age,
                        Users.Location
                    FROM (
                            SELECT
                                Name, UserID
                            FROM Badges
                            WHERE Name IN (
                                SELECT
                                    Name
                                FROM Badges
                                WHERE Class=1
                                GROUP BY Name
                                HAVING COUNT(*) BETWEEN 2 AND 10
                            )
                            AND Class=1
                        ) AS ValuableBadges
                    JOIN Users ON ValuableBadges.UserId=Users.Id
                  """, connection)

## <span style="color:peru">Pandas query</span>

In [21]:
Names=Badges.loc[Badges.Class==1,].groupby("Name").size().reset_index().rename(index=str,columns={0:"Count"})

Names=Names.loc[((Names.Count>1) & (Names.Count<11)),:]

ValuableBadges=Badges.loc[Badges.Class==1,("Id","UserId","Name")]

UserIdNames=pd.merge(Names,
                 ValuableBadges,
                 on="Name",how="inner").UserId.unique()

DF6_pd=Users.set_index("Id").loc[UserIdNames,("DisplayName","Reputation","Age","Location")].reset_index()

## Comparison

In [22]:

#pd.testing.assert_frame_equal(DF6_sql,DF6_pd) # fifth row is in diffrent line than it should so next ones are misplaced as well. i don't know why it's in different place 
np.equal(DF6_sql,DF6_pd)


,Id,DisplayName,Reputation,Age,Location
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,False,True
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,False,False,False
7,False,False,False,False,False
8,False,False,False,False,False
9,True,True,True,True,True
